## 한국어 BERT의 마스크드 언어 모델(Masked Language Model) 실습
사전 학습된 한국어 BERT를 이용하여 마스크드 언어 모델을 실습해봅시다. 이번 실습을 위해서만이 아니라 앞으로 사전 학습된 BERT를 사용할 때는 transformers라는 패키지를 자주 사용하게 됩니다. 실습 환경에 transformers 패키지를 설치해둡시다.
```
pip install transformers
```

### 1. 마스크드 언어 모델과 토크나이저
ransformers 패키지를 사용하여 모델과 토크나이저를 로드합니다. BERT는 이미 누군가가 학습해둔 모델을 사용하는 것이므로 우리가 사용하는 모델과 토크나이저는 항상 맵핑 관계여야 합니다. 예를 들어서 A라는 이름의 BERT를 사용하는데, B라는 이름의 BERT의 토크나이저를 사용하면 모델은 텍스트를 제대로 이해할 수 없습니다. A라는 BERT의 토크나이저는 '사과'라는 단어를 36번으로 정수 인코딩하는 반면에, B라는 BERT의 토크나이저는 '사과'라는 단어를 42번으로 정수 인코딩하는 등 단어와 맵핑되는 정수 정보 자체가 다르기 때문입니다.

klue/bert-base는 대표적인 한국어 BERT입니다. klue/bert-base의 마스크드 언어 모델과 klue/bert-base의 토크나이저를 로드해봅시다.

In [1]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

2022-04-12 10:39:57.390887: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-12 10:39:57.390994: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


TFBertForMaskedLM.from_pretrained('BERT 모델 이름')을 넣으면 [MASK]라고 되어있는 단어를 맞추기 위한 마스크드 언어 모델링을 위한 구조로 BERT를 로드합니다. 다시 말해서 BERT를 마스크드 언어 모델 형태로 로드합니다.
  
AutoTokenizer.from_pretrained('모델 이름')을 넣으면 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드합니다.

In [2]:
model = TFBertForMaskedLM.from_pretrained('bert-large-uncased')
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.37G [00:00<?, ?B/s]

2022-04-11 18:24:14.400932: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-11 18:24:14.400966: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-11 18:24:14.400980: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-41K4BMT): /proc/driver/nvidia/version does not exist
2022-04-11 18:24:14.403646: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBer

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

### 2. BERT의 입력
''Soccer is a really fun [MASK]'라는 임의의 문장이 있다고 해봅시다. 이를 마스크드 언어 모델의 입력으로 넣으면, 마스크드 언어 모델은 [MASK]의 위치에 해당하는 단어를 예측합니다. 마스크드 언어 모델의 예측 결과를 보기위해서 bert-large-uncased의 토크나이저를 사용하여 해당 문장을 정수 인코딩해봅시다.

In [3]:
inputs = tokenizer('Soccer is a really fun [MASK].', return_tensors='tf')

토크나이저로 변환된 결과에서 input_ids를 통해 정수 인코딩 결과를 확인할 수 있습니다.

In [4]:
print(inputs['input_ids'])

tf.Tensor([[ 101 4715 2003 1037 2428 4569  103 1012  102]], shape=(1, 9), dtype=int32)


토크나이저로 변환된 결과에서 token_type_ids를 통해서 문장을 구분하는 세그먼트 인코딩 결과를 확인할 수 있습니다.

In [5]:
print(inputs['token_type_ids'])

tf.Tensor([[0 0 0 0 0 0 0 0 0]], shape=(1, 9), dtype=int32)


현재의 입력은 문장이 두 개가 아니라 한 개이므로 여기서는 문장 길이만큼의 0 시퀀스를 얻습니다. 만약 문장이 두 개였다면 두번째 문장이 시작되는 구간부터는 1의 시퀀스가 나오게 되지만, 여기서는 해당되지 않습니다.
  
토크나이저로 변환된 결과에서 attention_mask를 통해서 실제 단어와 패딩 토큰을 구분하는 용도인 어텐션 마스크를 확인할 수 있습니다.

In [6]:
print(inputs['attention_mask'])

tf.Tensor([[1 1 1 1 1 1 1 1 1]], shape=(1, 9), dtype=int32)


현재의 입력에서는 패딩이 없으므로 여기서는 문장 길이만큼의 1 시퀀스를 얻습니다. 만약 뒤에 패딩이 있었다면 패딩이 시작되는 구간부터는 0의 시퀀스가 나오게 되지만, 여기서는 해당되지 않습니다. 좀 더 다양한 패턴의 입력은 뒤의 텍스트 분류, 개체명 인식, 질의 응답 실습에서 이어서 보겠습니다.

### 3. [MASK] 토큰 예측하기
FillMaskPipeline은 모델과 토크나이저를 지정하면 손쉽게 마스크드 언어 모델의 예측 결과를 정리해서 보여줍니다. FillMaskPipeline에 우선 앞서 불러온 모델과 토크나이저를 지정해줍니다.

In [7]:
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

이제 입력 문장으로부터 [MASK]의 위치에 들어갈 수 있는 상위 5개의 후보 단어들을 출력해봅시다.

In [8]:
pip('Soccer is a really fun [MASK].')

[{'score': 0.7621119618415833,
  'token': 4368,
  'token_str': 'sport',
  'sequence': 'soccer is a really fun sport.'},
 {'score': 0.20341996848583221,
  'token': 2208,
  'token_str': 'game',
  'sequence': 'soccer is a really fun game.'},
 {'score': 0.012208551168441772,
  'token': 2518,
  'token_str': 'thing',
  'sequence': 'soccer is a really fun thing.'},
 {'score': 0.0018630317645147443,
  'token': 4023,
  'token_str': 'activity',
  'sequence': 'soccer is a really fun activity.'},
 {'score': 0.0013354889815673232,
  'token': 2492,
  'token_str': 'field',
  'sequence': 'soccer is a really fun field.'}]

In [9]:
pip('The Avengers is a really fun [MASK].')

[{'score': 0.25628936290740967,
  'token': 2265,
  'token_str': 'show',
  'sequence': 'the avengers is a really fun show.'},
 {'score': 0.17284080386161804,
  'token': 3185,
  'token_str': 'movie',
  'sequence': 'the avengers is a really fun movie.'},
 {'score': 0.11107732355594635,
  'token': 2466,
  'token_str': 'story',
  'sequence': 'the avengers is a really fun story.'},
 {'score': 0.07248973101377487,
  'token': 2186,
  'token_str': 'series',
  'sequence': 'the avengers is a really fun series.'},
 {'score': 0.07046627253293991,
  'token': 2143,
  'token_str': 'film',
  'sequence': 'the avengers is a really fun film.'}]

In [10]:
pip('I went to [MASK] this morning.')

[{'score': 0.3573067784309387,
  'token': 2147,
  'token_str': 'work',
  'sequence': 'i went to work this morning.'},
 {'score': 0.23304471373558044,
  'token': 2793,
  'token_str': 'bed',
  'sequence': 'i went to bed this morning.'},
 {'score': 0.12845060229301453,
  'token': 2082,
  'token_str': 'school',
  'sequence': 'i went to school this morning.'},
 {'score': 0.062305811792612076,
  'token': 3637,
  'token_str': 'sleep',
  'sequence': 'i went to sleep this morning.'},
 {'score': 0.04695264622569084,
  'token': 2465,
  'token_str': 'class',
  'sequence': 'i went to class this morning.'}]